<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href="http://mng.bz/orYv">LLMをゼロから作成する</a>書籍の補足コード（<a href="https://sebastianraschka.com">Sebastian Raschka</a>著）<br>
<br>コードリポジトリ: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 指示データセット用の「受動態」エントリの作成

- このノートブックはOpenAIのGPT-4を使用して、以下の例に示すように指示データセット用の「受動態」エントリを作成します

```python
{  
   'instruction': 'Identify the verb in the following sentence',
   'input': 'The cat sleeps on the couch.',
   'output': 'The verb in the sentence is "sleeps."',
   'output_2': 'The sentence is "sleeps."'   #  <---- 新しく作成されたエントリ
}  
```

In [1]:
# pip install -r requirements-extra.txt

In [2]:
from importlib.metadata import version

pkgs = ["openai",  # OpenAI API
        "tqdm",    # Progress bar
       ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

openai version: 1.30.3
tqdm version: 4.65.0


## OpenAI APIのテスト

- まず、OpenAI APIが正しくセットアップされているかテストしましょう
- まだアカウントをお持ちでない場合は、https://platform.openai.com/ でアカウントを作成する必要があります
- GPT-4 APIは無料ではないため、アカウントに資金を移転する必要があることに注意してください（https://platform.openai.com/settings/organization/billing/overview を参照）
- このノートブックのコードを使用して約200の受動態エントリを作成するには、約$0.13（13セント）かかります

- まず、https://platform.openai.com/api-keys で見つけることができるOpenAI APIの秘密鍵を提供する必要があります
- この鍵を誰とも共有しないようにしてください  
- この秘密鍵（`"sk-..."`）をこのフォルダの`config.json`ファイルに追加してください

In [3]:
import json
from openai import OpenAI

# Load API key from a JSON file. 
# Make sure to replace "sk-..." with your actual API key from https://platform.openai.com/api-keys
with open("config.json", "r") as config_file:
    config = json.load(config_file)
    api_key = config["OPENAI_API_KEY"]

client = OpenAI(api_key=api_key)

- まず、APIが意図した通りに動作することを確認するために、簡単な例でAPIを試してみましょう:

In [ ]:
def run_chatgpt(prompt, client, model="gpt-4-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )
    return response.choices[0].message.content


# 入力を準備
sentence = "I ate breakfast"
prompt = f"Convert the following sentence to passive voice: '{sentence}'"
run_chatgpt(prompt, client)

## JSONエントリの作成

- 次に、変更したいファイルを読み込みます:

In [5]:
import json

json_file = "instruction-examples.json"

with open(json_file, "r") as file:
    json_data = json.load(file)
    
print("Number of entries:", len(json_data))

Number of entries: 200


- そして、OpenAI chat APIが正しく動作することを確認するために、まず小さなサンプルで試してみます:

In [6]:
for entry in json_data[:5]:
    text = entry["output"]
    prompt = f"Without adding any response or explanation, convert the following text to passive voice: {text}"
    
    print("\nInput:")
    print(">>", text)
    print("\nOutput:")
    print(">>", run_chatgpt(prompt, client))
    print("\n-------------------------")


Input:
>> The verb in the sentence is "sleeps."

Output:
>> The sentence is "sleeps."

-------------------------

Input:
>> The plural form of "goose" is "geese."

Output:
>> The plural form of "goose" is referred to as "geese."

-------------------------

Input:
>> The three primary colors are red, blue, and yellow.

Output:
>> Red, blue, and yellow are considered the three primary colors.

-------------------------

Input:
>> They had finished the game.

Output:
>> The game had been finished by them.

-------------------------

Input:
>> The abbreviation for "Doctor of Philosophy" is Ph.D.

Output:
>> The abbreviation "Ph.D." is used for "Doctor of Philosophy".

-------------------------


- 次に、生成されたエントリを`json_data`に追加し、プログレスバーを追加するようにコードを拡張しましょう:

In [ ]:
from tqdm import tqdm  # プログレスバーツール


for i, entry in tqdm(enumerate(json_data[:5]), total=len(json_data[:5])):
    text = entry["output"]
    prompt = f"Without adding any response or explanation, convert the following text to passive voice: {text}"
    json_data[i]["output_2"] = run_chatgpt(prompt, client)

- もう一度、新しいエントリ（`"output_2"`）が適切に見えることを確認しましょう

In [8]:
json_data[0]

{'instruction': 'Identify the verb in the following sentence: The cat sleeps on the couch.',
 'input': '',
 'output': 'The verb in the sentence is "sleeps."',
 'output_2': 'The sentence is "sleeps."'}

- 最後に、上記がすべて適切に見える場合は、JSONデータセット全体に対して受動態への変換を実行しましょう（約3分かかります）:

In [9]:
for i, entry in tqdm(enumerate(json_data), total=len(json_data)):
    text = entry["output"]
    prompt = f"Without adding any response or explanation, convert the following text to passive voice: {text}"
    json_data[i]["output_2"] = run_chatgpt(prompt, client)

100%|██████████████████████████████████████████████████████████████████| 200/200 [03:43<00:00,  1.12s/it]


- 変換が完了した後、ファイルを保存します:

In [10]:
new_json_file = json_file.replace(".json", "-modified.json")


with open(new_json_file, "w") as file:
    json.dump(json_data, file, indent=4)  # "indent" for pretty-printing